In [3]:
from typedspark import transform_to_schema
from typedspark import register_schema_to_dataset
from typedspark import Column, Schema, create_partially_filled_dataset
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql import SparkSession

spark = SparkSession.Builder().getOrCreate()


class Person(Schema):
    id: Column[IntegerType]
    name: Column[StringType]
    age: Column[IntegerType]


class Job(Schema):
    id: Column[IntegerType]
    salary: Column[IntegerType]


df_a = create_partially_filled_dataset(spark, Person, {Person.id: [1, 2, 3]})
df_b = create_partially_filled_dataset(spark, Job, {Job.id: [1, 2, 3]})

person = register_schema_to_dataset(df_a, Person)
job = register_schema_to_dataset(df_b, Job)


class PersonWithJob(Person, Job):
    pass


(
    transform_to_schema(
        df_a.join(
            df_b,
            person.id == job.id,
        ),
        PersonWithJob,
        {
            PersonWithJob.id: job.id,
        },
    ).show()
)

AnalysisException: [AMBIGUOUS_REFERENCE] Reference `id` is ambiguous, could be: [`id`, `id`].